# Purpose
# 目的



自2013年4月至2014年9月，中国各主流媒体报道了婴儿注射乙肝疫苗后死亡的事件，其中20例报道中公开了疫苗的批次。报道中的死婴涉及到三家公司： 深圳康泰， 大连汉信与天坛生物。中国药监局于2014年12月启动了调查，并在2014年1月3日和2014年1月17日，中国药监局两次发出通知，认定婴儿死亡与疫苗不存在关联。

从2014年9月至2018年8月，主流媒体没有再报告类似的死婴事件。

本项目试图从数据角度回答以下三个问题：

注射不同公司生产的疫苗，婴儿死亡几率是否相同？
注射同一公司不同时段生产的疫苗，婴儿死亡几率是否相同？

更多信息：https://zh.wikipedia.org/wiki/2013%E5%B9%B4%E4%B9%99%E8%82%9D%E7%96%AB%E8%8B%97%E6%AD%BB%E4%BA%A1%E4%BA%8B%E4%BB%B6


From Apr 2013 to Sep 2014, the mainsteam media in China reported infant death cases following hepatitis-B vaccinations, wherein 20 cases were reported with the batch numbers. these death cases involved three companies: Shenzhen Kangtai, Dalian Hanxin and Tiantan BioTech. Chinese Food and Drug Administration started the investigation from Dec 2013 and published the investigation results twice on Jan 3rd and Jan 17th in 2014 respectively which stated that no connections found between the hepatitis B vaccines and the death cases.

From Sep 2014 to Aug 2018, no death cases following hepatitis-B injections reported by the mainstream media.

This project tries to answer below questions:

1. If the infant motality rates are identical regardness which company produced the injected vaccine?
2. If the infant motality rates are identical regardness what time that the injected vaccine was produced?

more information (in Chinese):
https://zh.wikipedia.org/wiki/2013%E5%B9%B4%E4%B9%99%E8%82%9D%E7%96%AB%E8%8B%97%E6%AD%BB%E4%BA%A1%E4%BA%8B%E4%BB%B6


## Load Data
## 加载数据

从文件中加载死婴病例数据

Load infant death cases from file

In [133]:
import pandas as pd

df = pd.read_csv("../data/cases.txt", 
                 sep="|", 
                 header=None, 
                 names = ["city", "age_in_months", "batch_no", "company", "symptom"]
                )

display(df)

,city,age_in_months,batch_no,company,symptom
0,山东临沂郯城,1.0,201208021,天坛生物,不详
1,湖北鄂州,1.0,201208020,天坛生物,第2剂;鼻孔流血
2,山东菏泽,1.0,201208022,天坛生物,脸上都是奶，嘴唇黑紫没有气了
3,山东日照,2.0,201207081,大连汉信,"脑血栓,糖尿病"
4,内蒙古兴和县,0.0,C201204031,深圳康泰,第1剂;黄疸
5,四川,NaN,C201205049,深圳康泰,不详
6,广东顺德杏坛镇,1.0,C201205052,深圳康泰,第2剂;左额颞枕顶部硬膜下出血
7,广东广州,1.0,C201206058,深圳康泰,第1剂;头部水肿;颅内大量出血
8,宁夏银川,0.0,C201206059,深圳康泰,第1剂;嘴角发青;休克、呼吸衰竭、颅内出血
9,广西贺州,1.0,C201206069,深圳康泰,第2剂;呼吸困难


In [135]:
# load the dead cases
import collections

death_cases_dict = df.to_dict("records")

death_cases_count = {}
for case in death_cases_dict:
    batch = case["batch_no"]
    if death_cases_count.get(batch) == None:
        death_cases_count[batch] = 1    
    else:
        death_cases_count[batch] += 1    

In [136]:
kangtai = {} # data of Shenzhen Kangtai
hanxin = {}  # data of Dalian Hanxin
tiantan = {} # Data of Tiantan BioTech

# Initialize the data.
def init_data():
    for year in range(2010, 2019):
        for month in range(1, 13):
            year_month = "{}{:02d}".format(year, month)
            kangtai[year_month] = [0,0]
            hanxin[year_month] = [0,0]
            tiantan[year_month] = [0,0]

In [137]:
import re

def data_summarize(brand, month, production, deads):
    brand[month][0] += production
    brand[month][1] += deads

def get_month(chinese_date_str):
    year_month = re.sub("(\d{4})年(\d{1,2})月.*", r'\1,\2', chinese_date_str)
    year_month = year_month.split(',')
    return "{}{:02}".format(int(year_month[0]) - 3, int(year_month[1]))
    
init_data()
text_file = open("../data/hepatitis_b.txt", "r")
lines = text_file.readlines()
for line in lines:
    line = line.replace('\n', '')
    info = line.split(',')
    month = ""
    # get the production date which is expiration date minus 3 years.
    if info[7].find('#####') != -1:
        month = info[5][0:6]
    elif info[7].find('3/人份') != -1:
        month = get_month(info[9])
    else:
        month = get_month(info[7])

    batch_no = info[5]
    batch_no_list.append(batch_no)
    deaths = 0
    if death_cases_count.get(batch_no) != None:
        deaths = death_cases_count[batch_no]
    
    
    if line.find('康泰') != -1: # kangtai
        data_summarize(kangtai, month, int(info[6]), deaths)
    elif line.find('汉信') != -1: # hanxin
        data_summarize(hanxin, month, int(info[6]), deaths)
    elif line.find('天坛') != -1: # tiantan
        data_summarize(tiantan, month, int(info[6]), deaths)
        

        
print('kangtai:')
print(sorted(kangtai.items()))


#print('hanxin:')
#print(sorted(hanxin.items()))


#print('tiantan:')
#print(sorted(tiantan.items()))

 

kangtai:
[('201001', [0, 0]), ('201002', [0, 0]), ('201003', [0, 0]), ('201004', [0, 0]), ('201005', [0, 0]), ('201006', [0, 0]), ('201007', [0, 0]), ('201008', [0, 0]), ('201009', [0, 0]), ('201010', [0, 0]), ('201011', [0, 0]), ('201012', [0, 0]), ('201101', [0, 0]), ('201102', [0, 0]), ('201103', [0, 0]), ('201104', [0, 0]), ('201105', [0, 0]), ('201106', [0, 0]), ('201107', [0, 0]), ('201108', [0, 0]), ('201109', [0, 0]), ('201110', [0, 0]), ('201111', [0, 0]), ('201112', [0, 0]), ('201201', [0, 0]), ('201202', [2350047, 0]), ('201203', [4114968, 0]), ('201204', [3377928, 1]), ('201205', [3081564, 2]), ('201206', [4690848, 3]), ('201207', [5474712, 7]), ('201208', [4473901, 0]), ('201209', [4026876, 1]), ('201210', [3609660, 0]), ('201211', [4700814, 1]), ('201212', [3239234, 0]), ('201301', [706596, 0]), ('201302', [0, 0]), ('201303', [0, 0]), ('201304', [0, 0]), ('201305', [0, 0]), ('201306', [0, 0]), ('201307', [0, 0]), ('201308', [292852, 0]), ('201309', [0, 0]), ('201310', [0,

## 总体质量分析
## Overall Quality Analytics

本节的目的是分析: 无论注射哪家公司的疫苗，婴儿的死亡率是否相同。

The purpose of this section is to figure out if the infant motality rates were identical regardless which company produced the vaccine.

In [127]:
import numpy as np
from datetime import datetime, date, time
months=np.asarray(list(kangtai.keys()))

months_list = [datetime.strptime(month, "%Y%m").date() for month in months]
kangtai_values=np.asarray(list(kangtai.values()))
tiantan_values=np.asarray(list(tiantan.values()))
hanxin_values=np.asarray(list(hanxin.values()))


In [146]:
import scipy.misc
# find out if the qualities are different among the companies.

def calc_prob_helper(prob_dist, n, k):
    combination = scipy.special.factorial(n) / (scipy.special.factorial(k) * scipy.special.factorial(n - k))
    prob = prob_dist ** k * (1 - prob_dist) ** (n - k) * combination
    return prob

prod_sum = [np.sum(kangtai_values[:35, 0]), np.sum(tiantan_values[:35, 0]), np.sum(hanxin_values[:35, 0])]

cases_sum = [np.sum(kangtai_values[:35, 1]), np.sum(tiantan_values[:35, 1]), np.sum(hanxin_values[:35, 1])]

prod_sum_prob_dist = prod_sum / np.sum(prod_sum)
all_cases = np.sum(kangtai_values[:35, 1] + tiantan_values[:35, 1] + hanxin_values[:35, 1])
probility = calc_prob_helper(prod_sum_prob_dist, all_cases, cases_sum)

overview = {'Company': ['Kangtai', 'Tiantan', 'Hanxin'],
         'Production': prod_sum,
         'Percentage': prod_sum_prob_dist * 100,
         'Actually Death Cases': cases_sum,
          'Expected Death Cases': all_cases * prod_sum_prob_dist,
         'p-value': probility
        }
pd.DataFrame.from_dict(overview)


,Company,Production,Percentage,Actually Death Cases,Expected Death Cases,Probability
0,Kangtai,39901318,41.071098,15,7.803509,0.000746
1,Tiantan,9592869,9.874101,3,1.876079,0.176771
2,Hanxin,47657630,49.054800,1,9.320412,0.000050


In [144]:
# calculate the probability
import math

np.set_printoptions(precision=4)
def calc_prob(brand_data):
    # calculate the probability distribution among all months
    production_per_month = brand_data[:35,0]
    total_production = np.sum(production_per_month)
    prob_dist = production_per_month / total_production
    
    cases_per_month = brand_data[:35, 1]
    total_cases = np.sum(cases_per_month)
    case_ratio = total_cases / total_production
    prob = calc_prob_helper(prob_dist, total_cases, cases_per_month)
    return prob
    

kangtai_prob = calc_prob(kangtai_values)
tiantan_prob = calc_prob(tiantan_values)
hanxin_prob = calc_prob(hanxin_values)


In [145]:
import plotly.plotly as py
import plotly.graph_objs as go
from datetime import datetime, date, time

#plotly.tools.set_credentials_file(username='smartestrobotdai', api_key='n3hPxGClfzgBxeKxwrnF')

trace1 = go.Scatter(
    x=months_list,
    y=kangtai_values[:36,0],
    marker=dict(
        color = 'red', #set color equal to a variable
        showscale=True
    ),
    name='kangtai'
)
trace2 = go.Scatter(
    x=months_list,
    y=tiantan_values[:36,0],
    marker=dict(
        color = 'orange', #set color equal to a variable
        showscale=True
    ),
    name='tiantan'
)
trace3 = go.Scatter(
    x=months_list,
    y=hanxin_values[:36,0],
    marker=dict(
        color = 'green', #set color equal to a variable
        showscale=True
    ),
    name='hanxin'
)
trace4 = go.Bar(
    name='kangtai',
    x=months_list,
    y=kangtai_values[:,1],
    yaxis='y2',
    textposition = 'auto',
    text = [p if p != 1 else None for p in kangtai_prob],
    marker=dict(
        color = 'red', #set color equal to a variable
    ),
    showlegend=False
)
trace5 = go.Bar(
    name='tiantan',
    x=months_list,
    y=tiantan_values[:,1],
    yaxis='y2',
    textposition = 'auto',
    text = [p if p != 1 else None for p in tiantan_prob],
    marker=dict(
        color = 'orange', #set color equal to a variable
    ),
    showlegend=False
)
trace6 = go.Bar(
    name='hanxin',
    x=months_list,
    y=hanxin_values[:,1],
    textposition = 'auto',
    yaxis='y2',
    text = [p if p != 1 else None for p in hanxin_prob],
    marker=dict(
        color = 'green', #set color equal to a variable
    ),
    showlegend=False
)

updatemenus = list([
    dict(
         active=3,
         buttons=list([
            dict(label = 'kangtai',
                 method = 'update',
                 args = [{'visible': [True, False, False, True, False, False]}]),
            dict(label = 'tiantan',
                 method = 'update',
                 args = [{'visible': [False, True, False, False, True, False]}]),
            dict(label = 'hanxin',
                 method = 'update',
                 args = [{'visible': [False, False, True, False, False, True]}]),
            dict(label = 'all',
                 method = 'update',
                 args = [{'visible': [True, True, True, True, True, True]}]),
            ]),
            x=0.1,
            y=1.2
        ),
    dict(
         active=1,
         buttons=list([   
            dict(label = 'before Dec 2012',
                 method = 'update',
                 args = [{'x': [months_list[:36]] * 6, 
                          'y': [kangtai_values[:36,0], tiantan_values[:36,0], hanxin_values[:36,0], 
                                kangtai_values[:,1], tiantan_values[:,1], hanxin_values[:,1]]}
                ]),
            dict(label = 'all',
                 method = 'update',
                 args = [{'x': [months_list] * 6,
                          'y': [kangtai_values[:,0], tiantan_values[:,0], hanxin_values[:,0], 
                               kangtai_values[:,1], tiantan_values[:,1], hanxin_values[:,1]]}
                ]),
             ]),
             x=0.1,
             y=1.3
        ),
    ])


layout = go.Layout(
    title='Production(10μg) and Infants Death Cases',
    yaxis=dict(
        title='Monthly Production (doses)',
        domain=[0.55,1],
        exponentformat = "none"
    ),
    yaxis2=dict(
        title='Number of Death Cases',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        domain=[0,0.45],
        tickformat=".6d",
   
    ),
    

    updatemenus = updatemenus
)

data = [trace1, trace2, trace3, trace4, trace5, trace6]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='bar-line')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~smartestrobotdai/0 or inside your plot.ly account where it is named 'bar-line'


In [ ]:
现使用零假设方法来试图回答各公司的疫苗质量是否稳定。建立零假设：
零假设 h0: 选定某疫苗公司，接种疫苗的生产日期与婴儿的死亡率没有联系。
显著性水平α: 0.01。

从上图所示的p-value可以看出，深圳康泰在2012年7月，天坛生物在2012年8月所生产的疫苗，提高了婴儿的死亡率。

以上推论的置信区间在99% 以上。